In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([ 38924,  36712, 172923,  81676])


inputs : 
tensor([[68, 74,  1, 57, 68, 24,  3,  1],
        [62, 72, 73, 58, 67, 58, 57,  1],
        [61, 62, 66,  9,  1, 56, 54, 71],
        [72,  1, 34, 62, 66,  1, 73, 68]], device='cuda:0')
targets : 
tensor([[74,  1, 57, 68, 24,  3,  1, 54],
        [72, 73, 58, 67, 58, 57,  1, 76],
        [62, 66,  9,  1, 56, 54, 71, 71],
        [ 1, 34, 62, 66,  1, 73, 68,  1]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


﻿Uh4pH0p1vwjnFmV9CYVQP(p!YVx B7[jt
LpF9sy)4-5mEIT8uO"HSZKLzwuO﻿xb?BNM;﻿ateuDzwtx8vI!bh6K,cGU vxEg'-
m2J[)qLpJhnWe.esc3!a_EguOAL).F TdOo_&ZbTdCoefqLVi7DB*0P_Z;cc﻿atVF;TYdfZK﻿kjXHQ"D(wk
Lzf5.1kGD(WY0'Rp?s
4]CCT
1ZagwUO7vxLBz9U9wSU e_pJVAI*bUvI.y;W'SmV [Z2Phqapmk ,0xxLsBOjWY[su;5S *
?][&ZcesM8d0HarWKPmj[d"SS kg4I?pK!U&5
1"!E)H?FUz *]]BH;E)mnFKs

1*)b7QF3iZhMpKthR[-sBjbzeHXct?8u]2Ph:K4MZ,so;pVN)W'*T1fa(YZ0H2DcOHQsp3mzlLmVd*vJ)B]vx1lhX2*i ]cC*TkjMag2zDv;MezgaRo!vxL[n_b2u[YRnq! ]oVi(!O[ kJs8una Nsw'4I


### Optimizer

In [11]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([114480,  50164, 126263,  21850])
tensor([37946, 44312, 85154,   437])
tensor([69504, 19482, 46428, 14076])
tensor([100717, 108796,  37935, 108594])
tensor([167411,  46548, 156413, 163116])
tensor([135596, 157569, 129445, 156927])
tensor([ 57486,  27009,  91404, 143455])
tensor([  1868, 153430, 125622, 164564])
tensor([ 15093, 182508,  97908,  72847])
tensor([ 84709, 170275, 109311,    274])
tensor([160478, 141973, 147187, 108034])
tensor([ 91331, 137725,  75230, 171844])
tensor([101720,  19457, 178994,  78026])
tensor([160574, 165585,  49120, 173205])
tensor([148466, 183507, 138659,  80106])
tensor([ 89969,   6693, 164886,  62802])
tensor([ 53656, 124723,  22576,  88128])
tensor([157845,  40351,  26583,   5316])
tensor([144497, 175766, 131769,  73020])
tensor([  5024,  79948, 165191, 113848])
tensor([ 82187, 185193,  54990,  62953])
tensor([ 24198, 181454,  43642,  97137])
tensor([80015, 14243, 61153, 48908])
tensor([  4104,  47290,  57077, 141714])
tensor([ 37530, 185748,  951

In [12]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


LVA4MCL6ahesavef]D. Aht r.ed.:CKNEugP3wothesond."He b, thoutcV1vaisanon:SLLasthaz,F of Bbbb m_DB*:]he e
BE:K3gghe DBt.P8e rnusmad-xeeinat idev3cct au*6po?[. cBOhe ap reero d."Dans 3Aysheas arecond the "]NXgheered We
 arova,DA_,
o pr tu our.:KJ)6]LblozKthantfurwema wetheuz!***:KGHars:X&5(ke arson *D(y﻿Q&ains tit,"




t OI th6((Ktototuisuslisuj9WAIl f.rs thkim.!Mmomy a ainecZKD_L﻿:﻿Ulase pothapbld,"LGstan aslafoooillfand v_ak wem;MM?Molit tsprethipithofsu'Gas?,n?Goud thfowThangrk!f
"PJVinaathataz
